In [ ]:
import tensorflow as tf
from flow_models import *
from model_trainer import AnimeModel
from tensorflow.keras.mixed_precision import experimental as mixed_precision
import numpy as np
from pathlib import Path
import cv2

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [ ]:
frame_size = (960,540)
interp_ratio = [0.5]
model_f = hr_3_2_16
weight_dir = 'savedmodels/hr3216_real_flowcheck6/20'

In [ ]:
inputs = tf.keras.Input((frame_size[1],frame_size[0],6))
anime_model = AnimeModel(inputs, model_f, interp_ratio)
anime_model.load_weights(weight_dir)

In [ ]:
vid_dir = Path('data/cut')
vid_paths = [str(vid_dir/vn) for vn in os.listdir(vid_dir)]

In [ ]:
from tqdm.notebook import trange

In [ ]:
cap = cv2.VideoCapture(vid_paths[0])
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
writer = cv2.VideoWriter(f'{vid_paths[0]}_interp.mp4',fourcc,60,frame_size)
i = 0
ret, frame = cap.read()
for i in trange(600):
    if not cap.isOpened():
        break
    if ret:
        frame0 = frame
    else:
        break
    ret, _ = cap.read()
    if not ret:
        break
    
    ret, frame = cap.read()
    if ret:
        frame1 = frame
    else:
        break
    frame0_resized = cv2.resize(frame0, dsize=frame_size)
    frame1_resized = cv2.resize(frame1, dsize=frame_size)
    concated = np.concatenate([frame0_resized,frame1_resized],axis=-1).astype(np.float32)/ 255.0
    interped = anime_model(concated[np.newaxis,...])[0]
    interped = np.round(np.clip(interped, 0, 1) * 255).astype(np.uint8)
    writer.write(frame0_resized)
    writer.write(interped)


cap.release()
writer.release()